In [1]:
import os
import time

import numpy as np
import tensorflow as tf
from graph_nets import utils_tf
from sklearn.model_selection import train_test_split

from bclassification.utils_base import print_class_weights
from bclassification.utils_gns import (
    print_dataset,
    plot_metrics,
    plot_cm,
    plot_roc,
    describe_results,
    print_graph_dims,
)
from experience import load_experience
from lib.action_space import is_do_nothing_action
from lib.constants import Constants as Const
from lib.data_utils import make_dir, env_pf, extract_target_windows
from lib.dc_opf import TopologyConverter
from lib.gns import GraphNetwork
from lib.gns import (
    obses_to_lgraphs,
    lgraphs_to_cgraphs,
    get_graph_feature_dimensions
)
from lib.gns import tf_batched_graph_dataset
from lib.run_utils import create_logger
from lib.tf_utils import (
    ResidulaFCBlock,
    MatthewsCorrelationCoefficient,
)
from lib.visualizer import Visualizer, pprint

tf.keras.backend.set_floatx('float64')

Visualizer()

experience_dir = make_dir(os.path.join(Const.RESULTS_DIR, "performance-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "bc-gns"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))
create_logger(logger_name=f"{case_name}-{env_pf(env_dc)}", save_dir=case_results_dir)

case, collector = load_experience(case_name, agent_name, experience_dir, env_dc=env_dc)
obses, actions, rewards, dones = collector.aggregate_data()

pprint("    - Number of chronics:", dones.sum())
pprint("    - Observations:", len(obses))


L2RPN_2019_ART (dc)


--------------------------------------------------------------------------------
                                        Loading Experience
--------------------------------------------------------------------------------
    - Loading chronics:                 ./results/performance-aug/l2rpn_2019_art-dc/agent-mip-chronic-****
Chronic:                                0
        - O A R D:                      6913	6912	(6912,)	(6912,)
Chronic:                                1
        - O A R D:                      201	200	(200,)	(200,)
Chronic:                                3
        - O A R D:                      8065	8064	(8064,)	(8064,)
Chronic:                                4
        - O A R D:                      8065	8064	(8064,)	(8064,)
Chronic:                                7
        - O A R D:                      7777	7776	(7776,)	(7776,)
Chronic:                                10
        - O A R D:                      3104	3103	(3103,)	(3103,)
Chro

In [2]:
"""
    Parameters
"""
random_seed = 1

model_type = "gn"

n_window_targets = 0
n_window_history = 1
threshold = 0.5

dropout_rate = 0.1
n_hidden = 512
n_message_passes = 2

n_batch = 256
n_epochs = 500

downsampling_rate = 0.05

In [3]:
"""
    Datasets
"""

np.random.seed(random_seed)
tf.random.set_seed(random_seed)

labels = is_do_nothing_action(actions, case.env).astype(float)
pprint(
    "    - Labels:",
    f"{int(labels.sum())}/{labels.size}",
    "{:.2f} %".format(100 * labels.mean()),
)

mask_positive = extract_target_windows(labels, mask=~dones, n_window=n_window_targets)
mask_negative = np.logical_and(np.random.binomial(1, downsampling_rate, len(labels)), ~mask_positive)
mask_targets = np.logical_or(mask_positive, mask_negative)

pprint(
    "    - Mask (0):",
    mask_negative.sum(),
    "{:.2f} %".format(100 * mask_negative.sum() / mask_targets.sum()),
)
pprint(
    "    - Mask (1):",
    mask_positive.sum(),
    "{:.2f} %".format(100 * mask_positive.sum() / mask_targets.sum()),
)
pprint("    - Mask:", mask_targets.sum())

tc = TopologyConverter(case.env)
lgraphs_all = obses_to_lgraphs(obses, tc, n_window=n_window_history)
X_all = lgraphs_to_cgraphs(lgraphs_all)
Y_all = np.array(labels)

lgraphs = [lgraph for i, lgraph in enumerate(lgraphs_all) if mask_targets[i]]
X = lgraphs_to_cgraphs(lgraphs)
Y = Y_all[mask_targets]

lgraphs_train, lgraphs_test, Y_train, Y_test = train_test_split(
    lgraphs, Y, test_size=0.10, random_state=random_seed
)
lgraphs_train, lgraphs_val, Y_train, Y_val = train_test_split(
    lgraphs_train, Y_train, test_size=0.10, random_state=random_seed
)

X_train = lgraphs_to_cgraphs(lgraphs_train)
X_val = lgraphs_to_cgraphs(lgraphs_val)
X_test = lgraphs_to_cgraphs(lgraphs_test)

graph_dims = get_graph_feature_dimensions(lgraphs=lgraphs)
cgraph_dims = {**graph_dims, "n_nodes": tc.n_sub, "n_edges": 2 * tc.n_line}

print_dataset(X, Y, "Data")
print_dataset(X_train, Y_train, "Train")
print_dataset(X_val, Y_val, "Validation")
print_dataset(X_test, Y_test, "Test")
print_graph_dims(cgraph_dims)

    - Labels:                           3642/358424	1.02 %
    - Mask (0):                         17779	83.04 %
    - Mask (1):                         3630	16.96 %
    - Mask:                             21409
    - Data:                             X, Y	            (21409,)
        - X: globals                    21409	(40,)
        - X: edges                      21409	(40, 20)
        - X: nodes                      21409	(14, 66)
        - X: senders                    21409	(40,)
        - X: receivers                  21409	(40,)
        - Positive labels:              16.96 %
        - Negative labels:              83.04 %

    - Train:                            X, Y	            (17341,)
        - X: globals                    17341	(40,)
        - X: edges                      17341	(40, 20)
        - X: nodes                      17341	(14, 66)
        - X: senders                    17341	(40,)
        - X: receivers                  17341	(40,)
        - Positive labels: 

In [ ]:
"""
    Tensorflow datasets
"""

X_all_tf = tf_batched_graph_dataset(X_all, n_batch=n_batch, **graph_dims)
Y_all_tf = tf.data.Dataset.from_tensor_slices(Y_all).batch(n_batch)

X_train_tf = tf_batched_graph_dataset(X_train, n_batch=n_batch, **graph_dims)
Y_train_tf = tf.data.Dataset.from_tensor_slices(Y_train).batch(n_batch)

X_val_tf = tf_batched_graph_dataset(X_val, n_batch=n_batch, **graph_dims)
Y_val_tf = tf.data.Dataset.from_tensor_slices(Y_val).batch(n_batch)

X_test_tf = tf_batched_graph_dataset(X_test, n_batch=n_batch, **graph_dims)
Y_test_tf = tf.data.Dataset.from_tensor_slices(Y_test).batch(n_batch)

"""
    Signatures
"""

graphs_sig = utils_tf.specs_from_graphs_tuple(
    next(iter(X_train_tf)), dynamic_num_graphs=True
)
labels_sig = tf.TensorSpec(shape=[None], dtype=tf.dtypes.float64)

In [ ]:
"""
    Model
"""

class GraphNetworkBinary(tf.keras.Model):
    def __init__(
            self,
            graph_network,
            output_network,
            class_weight=None,
            metrics=(),
    ):
        super(GraphNetworkBinary, self).__init__()
        self.graph_network = graph_network
        self.output_network = output_network

        self.class_weight = dict()
        for c in class_weight:
            self.class_weight[c] = tf.constant(c, dtype=tf.float64)

        self.optimizer = tf.keras.optimizers.Adam(lr=1e-3)
        self.loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

        self.metrics_ = metrics

    def reset_metrics(self):
        for metric in self.metrics_:
            metric.reset_states()

    @tf.function(input_signature=[labels_sig, labels_sig])
    def update_metrics(self, y_true, y_pred):
        for metric in self.metrics_:
            if metric.name == "loss":
                metric.update_state(y_true, y_pred, sample_weight=self.sample_weight(y_true))
            else:
                metric.update_state(y_true, y_pred)

    @tf.function(input_signature=[graphs_sig, tf.TensorSpec(shape=(), dtype=tf.dtypes.bool)])
    def call(self, inputs, training=False):
        outputs = self.graph_network(inputs)

        edges = outputs.edges
        edges = tf.reshape(
            edges,
            shape=[-1, self.graph_network.n_edges, self.graph_network.n_edge_features]
        )
        edges = tf.math.reduce_max(edges, axis=1)

        nodes = outputs.nodes
        nodes = tf.reshape(
            nodes,
            shape=[-1, self.graph_network.n_nodes, self.graph_network.n_node_features]
        )
        nodes = tf.math.reduce_max(nodes, axis=1)

        outputs = tf.concat([nodes, edges, outputs.globals], axis=-1)
        outputs = self.output_network(outputs, training=training)

        return tf.reshape(outputs, [-1])

    @tf.function(input_signature=[graphs_sig, labels_sig])
    def train_step(self, x, y):
        with tf.GradientTape() as gt:
            probabilities = self(x, training=True)
            loss = self.compiled_loss_(y, probabilities)

        trainable_variables = list(self.graph_network.trainable_variables) + self.output_network.trainable_variables
        grads = gt.gradient(loss, trainable_variables)
        self.optimizer.apply_gradients(zip(grads, trainable_variables))

        self.update_metrics(y, probabilities)

        return loss, probabilities

    @tf.function(input_signature=[labels_sig])
    def sample_weight(self, y_true):
        sample_weight = None
        if self.class_weight:
            sample_weight = tf.multiply(1.0 - y_true, self.class_weight[0]) + tf.multiply(y_true, self.class_weight[1])
            sample_weight = tf.reshape(sample_weight, [1, -1])

        return sample_weight

    @tf.function(input_signature=[labels_sig, labels_sig])
    def compiled_loss_(self, y_true, y_pred):
        loss = self.loss(y_true, y_pred, sample_weight=self.sample_weight(y_true))
        return loss

    def predict(self, x):
        predictions = []

        for batch in x:
            probabilities = self(batch, training=False)
            predictions.append(probabilities)

        predictions = tf.concat(predictions, axis=0)
        return predictions

    def evaluate(self, x, y):
        self.reset_metrics()
        predictions = self.predict(x)
        self.update_metrics(y, predictions)

        output = self.metrics_dict()
        return output

    def metrics_dict(self):
        output = dict()
        for metric in self.metrics_:
            output[metric.name] = metric.result()
        return output

In [ ]:
n_negative, n_positive = np.bincount(Y.astype(int))
n = n_negative + n_positive

class_weight = {0: n / n_negative / 2.0, 1: n / n_positive / 2.0}
initial_bias = np.log([n_positive / n_negative])

print_class_weights(class_weight)
pprint("Initial bias:", "{:.4f}".format(float(initial_bias)))

metrics = [
    tf.keras.metrics.BinaryCrossentropy(name="loss"),
    tf.keras.metrics.TruePositives(thresholds=threshold, name="tp"),
    tf.keras.metrics.FalsePositives(thresholds=threshold, name="fp"),
    tf.keras.metrics.TrueNegatives(thresholds=threshold, name="tn"),
    tf.keras.metrics.FalseNegatives(thresholds=threshold, name="fn"),
    tf.keras.metrics.BinaryAccuracy(threshold=threshold, name="accuracy"),
    tf.keras.metrics.Precision(thresholds=threshold, name="precision"),
    tf.keras.metrics.Recall(thresholds=threshold, name="recall"),
    MatthewsCorrelationCoefficient(threshold=threshold, name="mcc"),
]

gn_model = GraphNetwork(
    n_hidden_global=[n_hidden],
    n_hidden_node=[n_hidden],
    n_hidden_edge=[n_hidden],
    dropout_rate=dropout_rate,
    n_message_passes=n_message_passes,
    **cgraph_dims,
)

input_dims = gn_model.n_global_features + gn_model.n_node_features + gn_model.n_edge_features
out_model = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(
            n_hidden, activation="relu",
            input_shape=(input_dims,)
        ),
        tf.keras.layers.Dropout(dropout_rate),
        ResidulaFCBlock(n_hidden, activation="relu"),
        tf.keras.layers.Dropout(dropout_rate),
        ResidulaFCBlock(n_hidden, activation="relu"),
        tf.keras.layers.Dropout(dropout_rate),
        tf.keras.layers.Dense(
            1,
            activation="sigmoid",
            bias_initializer=tf.keras.initializers.Constant(initial_bias),
        ),
    ]
)

model = GraphNetworkBinary(
    graph_network=gn_model,
    output_network=out_model,
    class_weight=class_weight,
    metrics=metrics,
)

model_dir = make_dir(os.path.join(case_results_dir, f"model-000-{model_type}"))
checkpoint_path = os.path.join(model_dir, "ckpts")
ckpt = tf.train.Checkpoint(model=model, optimizer=model.optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=5)

pprint("Model directory:", model_dir)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    pprint("Restoring checkpoint from:", ckpt_manager.latest_checkpoint)

In [ ]:
"""
    Training
"""

training = {
    "epochs": [],
}
for metric in metrics:
    training[metric.name] = []
    training["val_" + metric.name] = []

for epoch in range(n_epochs):
    start = time.time()

    for batch, (graph_batch, label_batch) in enumerate(tf.data.Dataset.zip((X_train_tf, Y_train_tf))):
        loss, batch_predictions = model.train_step(graph_batch, label_batch)

    train_metrics = model.metrics_dict()
    val_metrics = model.evaluate(X_val_tf, Y_val)

    training["epochs"].append(epoch)
    for m_name, m_value in train_metrics.items():
        training[m_name].append(m_value)
    for m_name, m_value in val_metrics.items():
        training["val_" + m_name].append(m_value)

    pprint("Epoch: {}/{}".format(epoch, n_epochs), f"{np.round(time.time() - start, 3)} s")

ckpt_save_path = ckpt_manager.save()
pprint(f"    - Saving checkpoint to:", ckpt_save_path)

In [ ]:
plot_metrics(training, Y_train, Y_val, save_dir=model_dir)

In [ ]:
"""
    Performance
"""

Y_train_pred = model.predict(X_train_tf).numpy()
Y_val_pred = model.predict(X_val_tf).numpy()
Y_test_pred = model.predict(X_test_tf).numpy()
Y_all_pred = model.predict(X_all_tf).numpy()

results_train = model.evaluate(X_train_tf, Y_train)
results_val = model.evaluate(X_val_tf, Y_val)
results_test = model.evaluate(X_test_tf, Y_test)
results_all = model.evaluate(X_all_tf, Y_all)

describe_results(results_train, Y_train, name="Train")
describe_results(results_val, Y_val, name="Validation")
describe_results(results_test, Y_test, name="Test")
describe_results(results_all, Y_all, name="All")

plot_cm(Y_train, Y_train_pred, "Training", save_dir=model_dir)
plot_cm(Y_val, Y_val_pred, "Validation", save_dir=model_dir)
plot_cm(Y_test, Y_test_pred, "Test", save_dir=model_dir)
plot_cm(Y_all, Y_all_pred, "All", save_dir=model_dir)

plot_roc(
    [
        ("Training", Y_train, Y_train_pred),
        ("Validation", Y_val, Y_val_pred),
        ("Test", Y_test, Y_test_pred),
        ("All", Y_all, Y_all_pred),
    ],
    save_dir=model_dir,
)